In [69]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [70]:
pt_low, pt_high = 1, 12
phi_v_low, phi_v_high = 1, 30
itype = 1

In [71]:
isigma = 2
N_centr = 6
N_options = 1
N_vtx_bins = 4
N_m_bins = 180
centralities = ["0-88%","0-20%","20-40%","40-60%","60-80%","80-93%"]
centrality = [0,9,0,1,2,3,4,5,6,7,8,9]
colors=[1,2,4,6,root.kGreen+2]
file_path="input/"
file_name="m_ee_Run14AuAU_sixth_test_19087_1078runs.root"
#"m_ee_Run14AuAU_fourth_test_19072_800runs.root"#"m_ee_Run14AuAU_first_test_19060_100runs.root"

In [72]:
iOption = 0
rebin = 1
new_m_array = array( 'd', [0.,0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.725,0.8,0.875,0.95,1.0,1.05,1.15,1.3,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,3.55,3.65,3.75,3.85,4.5])
N_new_bins = len(new_m_array)-1
print(N_new_bins)

35


In [73]:
hists_ep_FG_read, hists_ep_BG_read, hists_ee_FG_read, hists_ee_BG_read = [], [], [], []
hist_stat_read=[]
infile = root.TFile.Open(file_path+file_name, "read")

for iOpt in range(1,N_options+1):
    histo_name = [f"inv_mass_ee_DCA_"]
    histo_name_like = [f"delt_phi_ee_DCA_"]
    print(histo_name)
    hists_ep_FG_read_centr, hists_ep_BG_read_centr, hists_ee_FG_read_centr, hists_ee_BG_read_centr = [], [], [], []
    hist_stat_centr = []
    for icent in range(N_centr):
        hist_FG = root.TH2D(f"FG{icent}_{iOption}_BG",f"1my{icent}_{iOption}_BG",180,0,4.5,30,0,30)
        hist_FG.SetDirectory(root.nullptr)
        hist_BG = root.TH2D(f"BG{icent}_{iOption}_BG",f"1my{icent}_{iOption}_BG",180,0,4.5,30,0,30)
        hist_BG.SetDirectory(root.nullptr)
        hist_FG_like = root.TH2D(f"FG_Like{icent}_{iOption}_BG",f"1my{icent}_{iOption}_BG",180,0,4.5,30,0,30)
        hist_FG_like.SetDirectory(root.nullptr)
        hist_BG_like = root.TH2D(f"BG_like{icent}_{iOption}_BG",f"1my{icent}_{iOption}_BG",180,0,4.5,30,0,30)
        hist_BG_like.SetDirectory(root.nullptr)
        hist_stat = infile.Get("c00_z00_r00/PoolStatistics")
        hist_stat.Add(hist_stat,-1)
        hist_stat.SetDirectory(root.nullptr)
        for i in range(centrality[icent*2],centrality[icent*2+1]+1):
            for j in range(N_vtx_bins):
                for k in range(0,1):
                    dirname=f"c0{i}_z0{j}_r00/"
                    if i>9: dirname=f"c{i}_z0{j}_r00/"
                    hist_FG_local = infile.Get(dirname+histo_name[k]+"FG12")
                    hist_BG_local = infile.Get(dirname+histo_name[k]+"BG12")
                    hist_FG_like_local = infile.Get(dirname+histo_name_like[k]+"FG11")
                    hist_BG_like_local = infile.Get(dirname+histo_name_like[k]+"BG11")
                    hist_FG_like_local.Add(infile.Get(dirname+histo_name_like[k]+"FG22"))
                    hist_BG_like_local.Add(infile.Get(dirname+histo_name_like[k]+"BG22"))
                    eta_FG_porj = hist_FG_like_local.ProjectionX(f"FG_loc_{icent}_{i*20+j}",pt_low,pt_high,phi_v_low,phi_v_high)
                    eta_BG_porj = hist_BG_like_local.ProjectionX(f"BG_loc_{icent}_{i*20+j}",pt_low,pt_high,phi_v_low,phi_v_high)

                    left = eta_FG_porj.FindBin(1.1)
                    right = eta_FG_porj.FindBin(2.3)+1

                    scale = [1, eta_FG_porj.Integral(left,right)/(eta_BG_porj.Integral(left,right)+0.1), 1, 1]
                    hists_local = [hist_FG_local, hist_BG_local, hist_FG_like_local, hist_BG_like_local]
                    hists_to_add = [hist_FG, hist_BG, hist_FG_like, hist_BG_like]

                    for iz in range(phi_v_low, 1+phi_v_high):
                        for ix in range(1,1+N_m_bins):
                            for ihist in range(2):
                                content = 0    
                                for iy in range(pt_low,1+pt_high):
                                    content += hists_local[ihist].GetBinContent(ix,iy,iz)*scale[ihist]
                                hists_to_add[ihist].SetBinContent(ix,iz,hists_to_add[ihist].GetBinContent(ix,iz)+content)

                    #hist_FG.Add(hist_FG_local)
                    #hist_BG.Add(hist_BG_local,scale)
                    #hist_FG_like.Add(hist_FG_like_local)
                    #hist_BG_like.Add(hist_BG_like_local,scale)
                    hist_stat.Add(infile.Get(dirname+"PoolStatistics"))

        hists_ep_FG_read_centr.append(hist_FG)
        hists_ep_BG_read_centr.append(hist_BG)
        hists_ee_FG_read_centr.append(hist_FG_like)
        hists_ee_BG_read_centr.append(hist_BG_like)
        hist_stat_centr.append(hist_stat)
    hists_ep_FG_read.append(hists_ep_FG_read_centr)
    hists_ep_BG_read.append(hists_ep_BG_read_centr)
    hists_ee_FG_read.append(hists_ee_FG_read_centr)
    hists_ee_BG_read.append(hists_ee_BG_read_centr)
    hist_stat_read.append(hist_stat_centr)

infile.Close()

['inv_mass_ee_DCA_']


In [78]:
c5 = root.TCanvas("ccc33","ccc33",1600,900)
iOpt = 0
pt_bins=[1,1,2,2,3,3,4,4,5,6,7,12]
c5.Divide(3,2)
legends =[]
projs,projs_BG=[],[]
for icentr in range(N_centr):
    c5.cd(icentr+1)
    
    proj1 = hists_ep_FG_read[iOption][icentr].ProjectionY(f"px{icentr}{iOption}",1,180)
    Format_Hist_total(proj1,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.13,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,0.9)
    projbg = hists_ep_BG_read[iOption][icentr].ProjectionY(f"pxbg{icentr}{iOption}",1,180)
    Format_Hist_total(projbg,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.13,0.13,0.01,0.01,0.07,0.06,21,1.5,2,3,2,0.85,0.9)
    #projbg.Scale(proj1.GetMaximum()/(projbg.GetMaximum()+1))
    root.gPad.SetLogy(1)
    proj1.Draw("p")
    projbg.Draw("same p")
    proj1.GetXaxis().SetNdivisions(505)
    proj1.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    legends[-1].AddEntry(proj1,"FG","pl")
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

c5.Draw()
c5.Print("output/m_ee/m_ee_newpt_FG_BG.png")

In [75]:
c5 = root.TCanvas("ccc33","ccc33",1500,900)
iOpt = 0
rebin = 10
pt_bins=[1,1,2,2,3,3,4,4,5,6,7,12]
c5.Divide(3,2)
legends =[]
projs,projs_BG=[],[]
for icentr in range(N_centr):
    c5.cd(icentr+1)
    
    proj1 = hists_ep_FG_read[iOption][icentr].ProfileX()
    proj1.Rebin(rebin)
    #proj1.Scale(1./rebin/rebin)
    
    Format_Hist_total(proj1,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.13,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,0.9)
    #root.gPad.SetLogy(1)
    proj1.Draw("p")
    #proj.Draw("same p")
    proj1.GetXaxis().SetNdivisions(505)
    proj1.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.1,0.1,0.4,0.4,centralities[icentr]))
    #legends[-1].AddEntry(proj1,"FG","pl")
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

c5.Draw()
c5.Print("output/m_ee/m_ee_newpt_FG_BG.png")